In [43]:
!pip install fastai nbdev


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [14]:
from fastai.text.all import *
path = untar_data(URLs.IMDB)
files = get_text_files(path, folders = ['train', 'test', 'unsup'])
files

(#100000) [Path('/root/.fastai/data/imdb/test/neg/6201_1.txt'),Path('/root/.fastai/data/imdb/test/neg/11881_2.txt'),Path('/root/.fastai/data/imdb/test/neg/11690_1.txt'),Path('/root/.fastai/data/imdb/test/neg/7300_1.txt'),Path('/root/.fastai/data/imdb/test/neg/2931_1.txt'),Path('/root/.fastai/data/imdb/test/neg/11165_4.txt'),Path('/root/.fastai/data/imdb/test/neg/1026_2.txt'),Path('/root/.fastai/data/imdb/test/neg/11499_3.txt'),Path('/root/.fastai/data/imdb/test/neg/3201_1.txt'),Path('/root/.fastai/data/imdb/test/neg/429_3.txt')...]

In [15]:
txts = L(o.open().read() for o in files[:2000])
txts[0]

"I didn't know what to expect when I started watching this movie, by the end of it I was pulling my hairs out. This was one of the most pathetic movies of this year...in fact, in the last ten years. David Dhawan should just give up his career as a director. I am yet to come across one original script that David Dhawan has worked on. This one was a complete bit y bit rip off Hitch. I have nothing against remakes as such, but this one is just so lousy that it makes you even hate the original one (which was pretty decent). I fail to understand what actors like Salman and Govinda saw in this script. I read somewhere, that this was supposed to be Govinda's comeback vehicle. If thats true, then only God can save his career. Salman just overacted to the hilt. Govinda who I think is an actor of very high caliber was completely wasted. Katrina Kaif and LAra Dutta had nothing to do apart form wearing designer clothes and smiling for no rhyme or reason. Please stay away form this one!"

In [4]:
spacy = WordTokenizer()
tkz = Tokenizer(spacy)

In [5]:
toks200 = txts[:200].map(tkz)
toks200[0]

(#231) ['xxbos','i','did',"n't",'know','what','to','expect','when','i'...]

In [6]:
num = Numericalize()
num.setup(toks200)
coll_repr(num.vocab,20)

"(#1880) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','the','.',',','a','of','and','to','is','it','i','this'...]"

In [7]:
nums200 = toks200.map(num)
dl = LMDataLoader(nums200)
x,y = first(dl)
x.shape,y.shape

(torch.Size([64, 72]), torch.Size([64, 72]))

In [8]:
# verify our X variable (should end with second last)
' '.join(num.vocab[o] for o in x[0][:20])

"xxbos i did n't know what to expect when i started watching this movie , by the end of it"

In [9]:
# verify our Y variable (should start from second token, and end with last)
' '.join(num.vocab[o] for o in y[0][:20])

"i did n't know what to expect when i started watching this movie , by the end of it i"

In [10]:
def tokenize_numericalize(txts):
    # choose a tokenizer
    spacy = WordTokenizer()
    tkz = Tokenizer(spacy)
    # tokenize the first 200 reviews
    tokens = txts.map(tkz)
    # numericalize
    numericalized = tokens.map(num)
    return tokens.map(num)


# process the first 200 reviews
numericalized = tokenize_numericalize(txts[:200])

# feed that to the Language Model Data Loader
dl = LMDataLoader(numericalized)
x,y = first(dl)
x.shape,y.shape

(torch.Size([64, 72]), torch.Size([64, 72]))

# Training a Text Classifier

numericalization can be done automatically when using `TextBlock`, it's also optimal as it:
- save the tokenized documents in a tmp folder, so it doesn't have to tokenize them more than once
- It runs multiple tokenization process in parallel, to take advcantage of your CPU

In [6]:
path

Path('/root/.fastai/data/imdb')

In [7]:
text_block = TextBlock.from_folder(path, is_lm=True)

In [20]:
get_imdb = partial(get_text_files, folders=['train', 'test', 'unsup'])

block = DataBlock(
    blocks=TextBlock.from_folder(path, is_lm=True),
    get_items=get_imdb, splitter=RandomSplitter(0.1)
)

block.summary(path, bs=128, seq_len=80)

Setting-up type transforms pipelines
Found 100000 items
2 datasets of sizes 90000,10000
Setting up Pipeline: Tokenizer -> Numericalize

Building one sample
  Pipeline: Tokenizer -> Numericalize
    starting from
      /root/.fastai/data/imdb/train/pos/6659_10.txt
    applying Tokenizer gives
      ['xxbos', 'xxmaj', 'it', 'takes', 'guts', 'to', 'make', 'a', 'movie', 'on', 'xxmaj', 'gandhi', 'in', 'xxmaj', 'india', ',', 'in', 'which', 'he', 'is', 'not', 'shown', 'as', 'a', 'man', 'who', 'could', 'do', 'no', 'wrong.this', 'movie', 'shows', 'how', 'a', 'xxmaj', 'mahatma', 'failed', 'to', 'be', 'a', 'decent', 'father(at', 'least', 'in', 'the', 'eyes', 'of', 'his', 'son', ')', '.', '\n\n', 'xxmaj', 'the', 'performances', 'are', 'terrific', ',', 'the', 'cinematography', 'fantastic', ',', 'the', 'direction', 'fabulous', ',', 'but', 'the', 'film', 'drags.if', 'the', 'intention', 'was', 'to', 'make', 'this', 'movie', 'without', 'any', 'box', '-', 'office', 'expectations', ',', 'which', 'i', 'as

RuntimeError: Error when trying to collate the data into batches with fa_collate, at least two tensors in the batch are not the same size.

Mismatch found on axis 0 of the batch and is of type `TensorText`:
	Item at index 0 has shape: torch.Size([239])
	Item at index 1 has shape: torch.Size([254])

Please include a transform in `after_item` that ensures all data of type TensorText is the same size

In [12]:
# on't forget about DataBlock's handy summary method, which is very useful for debugging data issues.

In [18]:
dls_lm = block.dataloaders(path, path=path, bs=128, seq_len=80)
dls_lm.show_batch(max_n=3)

,text,text_
0,"xxbos xxmaj its not the cast . a finer group of actors , you could not find . xxmaj its not the setting . xxmaj the director is in love with xxmaj new xxmaj york xxmaj city , and by the end of the film , so are we all ! xxmaj woody xxmaj allen could not improve upon what xxmaj bogdonovich has done here . xxmaj if you are going to fall in love , or find love ,","xxmaj its not the cast . a finer group of actors , you could not find . xxmaj its not the setting . xxmaj the director is in love with xxmaj new xxmaj york xxmaj city , and by the end of the film , so are we all ! xxmaj woody xxmaj allen could not improve upon what xxmaj bogdonovich has done here . xxmaj if you are going to fall in love , or find love , xxmaj"
1,"looking at this film then either borrow it for free xxrep 3 ! or rent it on your unlimited rental plan at your nearby video store . xxmaj do n't pay money to watch this xxrep 3 ! i wonder if they will take a return if i say the disc do n't work ? \n\n xxmaj basically the story line is 4 college students get recruited to work on an independent film . xxmaj during the filming process ,","at this film then either borrow it for free xxrep 3 ! or rent it on your unlimited rental plan at your nearby video store . xxmaj do n't pay money to watch this xxrep 3 ! i wonder if they will take a return if i say the disc do n't work ? \n\n xxmaj basically the story line is 4 college students get recruited to work on an independent film . xxmaj during the filming process , things"
2,"but it 's not half bad . \n\n xxmaj still waiting for ' surf 3 ' . xxbos xxmaj this has got to be the funniest movie i have seen in forever . xxmaj xxunk xxmaj guest is truly talented . xxmaj he has a gift for humor . i almost died laughing . xxmaj actually , when i saw this in theaters , i considered walking out because the movie was so dumb . xxmaj but it is dumb","it 's not half bad . \n\n xxmaj still waiting for ' surf 3 ' . xxbos xxmaj this has got to be the funniest movie i have seen in forever . xxmaj xxunk xxmaj guest is truly talented . xxmaj he has a gift for humor . i almost died laughing . xxmaj actually , when i saw this in theaters , i considered walking out because the movie was so dumb . xxmaj but it is dumb in"


In [14]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3, 
    metrics=[accuracy, Perplexity()]).to_fp16()

__Before fine-tuning let's see how it performs at generating a movie review so that we can compare after fine tuning to the imdb corpus__

In [16]:
def predict_next(txt, learn):
    N_WORDS = 40
    N_SENTENCES = 2
    preds = [learn.predict(txt, N_WORDS, temperature=0.75) 
             for _ in range(N_SENTENCES)]
    print("\n".join(preds))

predict_next("I liked this movie because", learn)

i liked this movie because of its European subject , which was also a popular film . In France , Henri François Basie 's Les Breton Le Mans ( French : La
i liked this movie because it was a remake of the Mazes of Swords . While several other characters of Jaws have been adapted for English audiences , the novel was not adapted into the film . The film


__What can you observe?__

Wrong:
- capitalization
- punctuation
- cut-off sentences

### Fine-Tuning the Language Model

Just like `vision_learner`, `language_model_learner` automatically calls `freeze` when using a pretrained model.

So this will only train the new weights, the new embeddings that are in our IMDb vocab, buit not in the pretrained model vocab.

In [17]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.998230,3.898461,0.300623,49.326469,13:35


In [18]:
learn.save('1epoch')

Path('/root/.fastai/data/imdb/models/1epoch.pth')

In [19]:
learn = learn.load('1epoch')
learn.unfreeze()
learn.fit_one_cycle(10, 2e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.765704,3.759862,0.316251,42.942509,14:15
1,3.715375,3.695756,0.323597,40.275997,14:06
2,3.629537,3.651433,0.328655,38.529819,14:05
3,3.556274,3.618382,0.332256,37.277214,14:13
4,3.490726,3.595954,0.335296,36.450455,14:23
5,3.413398,3.579280,0.337563,35.847736,14:06
6,3.343958,3.574382,0.339059,35.672581,14:08
7,3.305039,3.567643,0.340169,35.432976,14:10
8,3.244978,3.570725,0.340391,35.542358,14:03
9,3.224113,3.575514,0.340167,35.712982,14:10


In [20]:
learn.save_encoder('finetuned')

### Fined Tuned Text Generation

Let's see how well this imdb-fine_tuned model performs at predicting the next word of a movie review now

In [21]:
predict_next("I liked this movie because", learn)

i liked this movie because it was so gripping . i was a teenager at the time . The story was very good , and the acting was top notch . It was very realistic and it was told in a sophisticated way
i liked this movie because it is a very good movie . This is not quite the kind of movie that the director has made with a very mature and very well written script . This movie has not only a very emotional


__what can you observe?__


good:
- Seems like the content seems better
  
wrong:
- punctation
- capitalization

### Creating the Classifier DataLoaders - Dataloader defines the task!

- `TextBlock.from_folder` no longer has the `is_lm=True` parameter.
- We pass the vocab we created for the language model fine-tuning.

By passing is_lm=False (or not passing is_lm at all, since it defaults to False) we tell TextBlock that we have regular labeled data, rather than using the next tokens as labels.


In [24]:
dls_clas = DataBlock(
    blocks=(TextBlock.from_folder(path, vocab=dls_lm.vocab),CategoryBlock),
    get_y = parent_label,
    get_items=partial(get_text_files, folders=['train', 'test']),
    splitter=GrandparentSplitter(valid_name='test')
).dataloaders(path, path=path, bs=128, seq_len=72)
     

In [25]:
dls_clas.show_batch(max_n=3)

,text,category
0,"xxbos xxmaj match 1 : xxmaj tag xxmaj team xxmaj table xxmaj match xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley vs xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley started things off with a xxmaj tag xxmaj team xxmaj table xxmaj match against xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit . xxmaj according to the rules of the match , both opponents have to go through tables in order to get the win . xxmaj benoit and xxmaj guerrero heated up early on by taking turns hammering first xxmaj spike and then xxmaj bubba xxmaj ray . a xxmaj german xxunk by xxmaj benoit to xxmaj bubba took the wind out of the xxmaj dudley brother . xxmaj spike tried to help his brother , but the referee restrained him while xxmaj benoit and xxmaj guerrero",pos
1,"xxbos * * attention xxmaj spoilers * * \n\n xxmaj first of all , let me say that xxmaj rob xxmaj roy is one of the best films of the 90 's . xxmaj it was an amazing achievement for all those involved , especially the acting of xxmaj liam xxmaj neeson , xxmaj jessica xxmaj lange , xxmaj john xxmaj hurt , xxmaj brian xxmaj cox , and xxmaj tim xxmaj roth . xxmaj michael xxmaj canton xxmaj jones painted a wonderful portrait of the honor and dishonor that men can represent in themselves . xxmaj but alas … \n\n it constantly , and unfairly gets compared to "" braveheart "" . xxmaj these are two entirely different films , probably only similar in the fact that they are both about xxmaj scots in historical xxmaj scotland . xxmaj yet , this comparison frequently bothers me because it seems",pos
2,"xxbos xxmaj titanic directed by xxmaj james xxmaj cameron presents a fictional love story on the historical setting of the xxmaj titanic . xxmaj the plot is simple , xxunk , or not for those who love plots that twist and turn and keep you in suspense . xxmaj the end of the movie can be figured out within minutes of the start of the film , but the love story is an interesting one , however . xxmaj kate xxmaj winslett is wonderful as xxmaj rose , an aristocratic young lady betrothed by xxmaj cal ( billy xxmaj zane ) . xxmaj early on the voyage xxmaj rose meets xxmaj jack ( leonardo dicaprio ) , a lower class artist on his way to xxmaj america after winning his ticket aboard xxmaj titanic in a poker game . xxmaj if he wants something , he goes and gets it",pos


In [27]:
nums_samp = toks200[:10].map(num)
nums_samp.map(len)

(#10) [231,260,148,335,137,363,155,299,145,60]

### Using our fine-tuned model (encoder)

In [28]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, 
                                metrics=accuracy).to_fp16()

learn = learn.load_encoder('finetuned')

### Gradually Unfreezing the Classifier

The last step is to train with discriminative learning rates and gradual unfreezing. In computer vision we often unfreeze the model all at once, but for NLP classifiers, we find that unfreezing a few layers at a time makes a real difference:`

In [30]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.241581,0.180199,0.930840,00:24


We can pass -2 to freeze_to to freeze all except the last two parameter groups:

In [35]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.178601,0.149430,0.945400,00:28


Then we can unfreeze a bit more, and continue training:

In [36]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.160318,0.148327,0.947840,00:35


And finally, the whole model!

In [37]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.127381,0.151703,0.945080,00:43
1,0.110951,0.153922,0.946520,00:43


We reached 94.3% accuracy, which was state-of-the-art performance just three years ago. 

In [54]:
learn.export('/workspace/imdb_classifier.pkl')